In [ ]:
import os
import pandas as pd
import sqlite3

#### Combine downloaded data into one dataframe
In order to facilitate analysis, I have combined all the data into a single file with the data points I am interested in. This way, I can just load the data in the future, rather than having to combine all the data every time.

In [ ]:
conn = sqlite3.connect(':memory:')

# Define the root directory containing the folders
root_dir = '.\\'

# Initialize an empty list to store the DataFrames
dataframes = []

# Loop through each folder in the root directory
for folder_name in os.listdir():
    if '.' in folder_name:
        continue
    folder_path = os.path.join(folder_name)
    
    # Ensure the current path is a directory
    if os.path.isdir(folder_path):
        # Define the paths to the CSV files
        nibrs_file_path = os.path.join(folder_path, 'NIBRS_ARRESTEE.csv')
        ref_race_file_path = os.path.join(folder_path, 'REF_RACE.csv')

        # Check if both files exist
        if os.path.exists(nibrs_file_path) and os.path.exists(ref_race_file_path):
            # Read the CSV files
            nibrs_df = pd.read_csv(nibrs_file_path)
            ref_race_df = pd.read_csv(ref_race_file_path)
            
            # Turn dataframes into sql tables for joining
            nibrs_df.to_sql('table1', conn, index=False, if_exists='replace')
            ref_race_df.to_sql('table2', conn, index=False, if_exists='replace')
            
            # Join the dataframes 
            query = """
            SELECT *
            FROM table1
            JOIN table2
            ON table1.RACE_ID = table2.RACE_ID
            """
            joined_df = pd.read_sql_query(query, conn)

            # Add a new column with the state abbreviation
            joined_df['State'] = folder_name
            
            # Append the joined dataframe to the list
            dataframes.append(joined_df)

# Concatenate all DataFrames in the list into a single DataFrame
final_df = pd.concat(dataframes, ignore_index=True)

# Save the concatenated DataFrame to a new CSV file
output_file_path = os.path.join(root_dir, 'arrest_data_USA.csv')
final_df.to_csv(output_file_path, index=False)

print(f'Concatenated data saved to {output_file_path}')

conn.close()

#### Basic Data Analysis
List basic data about the file, which should help to dive deeper into various components in the future. 

In [ ]:
#Analyze the resulting DataFrame using pandas
print("\nDescriptive Statistics of the Joined DataFrame:")
print(joined_df.describe())

#### Load data on state demographics in the analysed year (2022)
This data was retrieved based off estimate from the USA government census. Here is the URL:
https://www.census.gov/data/tables/time-series/demo/popest/2020s-state-detail.html#v2022

Category options are described here:
https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2020-2022/sc-est2022-alldata6.pdf

In [ ]:
demographic_df = pd.read_csv('.\\DemographicDataPerState.csv')
print(demographic_df.describe())

In [ ]:
demographic_df.head()

Grab a list of all the states from the demographic data so that they can:
1. Be combined
2. Be used to query for specific data using a loop

In [ ]:
conn = sqlite3.connect(':memory:')

#Turn demographic data into a SQL table 
demographic_df.to_sql('demo_table', conn, index=False, if_exists='replace')

query = """
SELECT DISTINCT NAME
FROM demo_table"""

state_names_df = pd.read_sql_query(query, conn)

conn.close()

In [ ]:
#Turn state names into a numpy array for some looping
state_names_array = state_names_df.to_numpy()

#### Query for Race based demographic data per state
This will allow me to break down the arrest data while being congisant of how different states may have different demographic data. I.e. if Alabama arrests 4 black people, but only has 4, that is way more significant than NY arresting 4 if they have 4000.

In [ ]:
conn = sqlite3.connect(':memory:')
#create a list of all the dataframes to compare the demographics of a state to its arrests
state_demo = []

#Turn demographic data into a SQL table 
demographic_df.to_sql('demo_table', conn, index=False, if_exists='replace')

for i in range(len(state_names_array)):
    
    query = f"""
    SELECT RACE, SUM(POPESTIMATE2022) * 100.0 / (SELECT SUM(POPESTIMATE2022) FROM demo_table WHERE NAME LIKE '{state_names_array[i][0]}') as "Population Percentage"
    FROM demo_table
    WHERE NAME LIKE '{state_names_array[i][0]}' 
    GROUP BY RACE"""

    #create a temporary so a column can be added
    temp_df = pd.read_sql_query(query, conn)
    
    #add said column
    temp_df['State'] = state_names_array[i][0]
    
    #add all dataframes to one group
    state_demo.append(temp_df)
    
    #delete temp_df so no leaking anything
    del temp_df

    
#Grab data for the country as a whole as well

query = """
SELECT RACE, SUM(POPESTIMATE2022) * 100.0 / (SELECT SUM(POPESTIMATE2022) FROM demo_table) as "Population Percentage"
FROM demo_table 
GROUP BY RACE"""

#create a temporary so a column can be added
temp_df = pd.read_sql_query(query, conn)

#add said column
temp_df['State'] = "Total"

#add all dataframes to one group
state_demo.append(temp_df)

#delete temp_df so no leaking anything
del temp_df

conn.close()

# Concatenate all DataFrames in the list into a single DataFrame
demo_df = pd.concat(state_demo, ignore_index=True)

In [ ]:
demo_df.to_csv("demogaphic_data.csv",index=False)